This notebook shows sql clustering using embedding and vector database

## 1. Breparing Database

In [13]:
%pip install -r requirements.txt

     ---------------------------------------- 0.0/647.5 kB ? eta -:--:--
      --------------------------------------- 10.2/647.5 kB ? eta -:--:--
      --------------------------------------- 10.2/647.5 kB ? eta -:--:--
     - ----------------------------------- 30.7/647.5 kB 259.2 kB/s eta 0:00:03
     - ----------------------------------- 30.7/647.5 kB 259.2 kB/s eta 0:00:03
     -- ---------------------------------- 41.0/647.5 kB 195.7 kB/s eta 0:00:04
     ---- -------------------------------- 81.9/647.5 kB 305.0 kB/s eta 0:00:02
     ----- ------------------------------- 92.2/647.5 kB 308.0 kB/s eta 0:00:02
     ------ ----------------------------- 122.9/647.5 kB 359.9 kB/s eta 0:00:02
     ------- ---------------------------- 143.4/647.5 kB 387.0 kB/s eta 0:00:02
     ---------- ------------------------- 194.6/647.5 kB 452.9 kB/s eta 0:00:02
     ------------- ---------------------- 245.8/647.5 kB 518.8 kB/s eta 0:00:01
     --------------- -------------------- 286.7/647.5 kB 55

  DEPRECATION: annoy is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:

import pandas as pd

splits = {'train': 'synthetic_text_to_sql_train.snappy.parquet', 'test': 'synthetic_text_to_sql_test.snappy.parquet'}
df = pd.read_parquet("hf://datasets/gretelai/synthetic_text_to_sql/" + splits["train"])

df.head

<bound method NDFrame.head of           id                 domain  \
0       5097               forestry   
1       5098       defense industry   
2       5099         marine biology   
3       5100     financial services   
4       5101                 energy   
...      ...                    ...   
99995  89651              nonprofit   
99996  89652                 retail   
99997  89653       fitness industry   
99998  89654      space exploration   
99999  89655  wildlife conservation   

                                      domain_description    sql_complexity  \
0      Comprehensive data on sustainable forest manag...       single join   
1      Defense contract data, military equipment main...       aggregation   
2      Comprehensive data on marine species, oceanogr...         basic SQL   
3      Detailed financial data including investment s...       aggregation   
4      Energy market data covering renewable energy s...  window functions   
...                              

In [2]:
from transformers import pipeline
pipe = pipeline("feature-extraction", model="microsoft/codebert-base")     

c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cpu


In [7]:
import numpy as np


sql_query = "SELECT name, age FROM users WHERE age > 30"
embeddings = pipe(sql_query)

embedding_size = np.array(embeddings).shape
print(f"Embedding size: {embedding_size}")

Embedding size: (1, 12, 768)


# Using FAISS as Vector database

In [10]:
from annoy import AnnoyIndex

f = 768  # 向量维度

t = AnnoyIndex(f, 'angular')  # 使用 angular 距离度量


ModuleNotFoundError: No module named 'faiss'